In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN,LSTM,Bidirectional,GRU,Dropout, TextVectorization, Embedding, Input
from keras.regularizers import l2

upload dataset

In [2]:
filepath = r"/content/judge-1377884607_tweet_product_company.csv"

data = pd.read_csv(filepath,usecols=  ['tweet_text', 'is_there_an_emotion_directed_at_a_brand_or_product'], encoding='ISO 8859-1')

In [3]:
pd.set_option('display.max_colwidth', None)
data.sample(10)

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
2492,Why should/shouldn't I get an iPad 2? Discuss :-) #SXSW,No emotion toward brand or product
932,"No Droid? Not surprised. RT @mention spotted at #sxsw: table of three people with 8 phones, an iPad and a MacBook out. #gadgetoverload",No emotion toward brand or product
8701,Record your #sxsw experience with your iPhone.\n{link},No emotion toward brand or product
4353,"Google's Mobile Future, and the Elusive 'Power of Here&quot; #SXSW Interactive @mention",No emotion toward brand or product
8061,"@mention Soon, in fact, at #SXSW you can play with it on an iPad (at our booth). Are you at the convention?",Positive emotion
2065,First 19 Apps To Download For Your #iPad 2 {link} #SXSW #in via @mention @mention,No emotion toward brand or product
6832,RT @mention Updated NPR Music iPhone app has song info for All Songs 24/7 &amp; live video streaming just in time for #SXSW {link},Positive emotion
4606,My tweeting from #sxsw been pretty non existent today thanks to the @mention iPhone app #fail - Hootsuite is the epic replacement! #owllove,Negative emotion
2784,Getting ready for my iPad 2 vs Android panel at #SXSW come check it out at 230!,No emotion toward brand or product
7794,Google party broke up. At the Tumblr / Funny or Die party. Fun! #sxsw,No emotion toward brand or product


In [4]:
data.shape

(9093, 2)

In [5]:
data.columns = ['tweet', 'emotion']

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   tweet    9092 non-null   object
 1   emotion  9093 non-null   object
dtypes: object(2)
memory usage: 142.2+ KB


In [7]:
data.head()

,tweet,emotion
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [8]:
data.isna().sum()

,0
tweet,1
emotion,0


In [9]:
print(data[data['tweet'].isna() == True])

data = data.dropna(ignore_index=True)

data[0:10]

  tweet                             emotion
6   NaN  No emotion toward brand or product


,tweet,emotion
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd,No emotion toward brand or product
6,"#SXSW is just starting, #CTIA is around the corner and #googleio is only a hop skip and a jump from there, good time to be an #android fan",Positive emotion
7,Beautifully smart and simple idea RT @madebymany @thenextweb wrote about our #hollergram iPad app for #sxsw! http://bit.ly/ieaVOB,Positive emotion
8,Counting down the days to #sxsw plus strong Canadian dollar means stock up on Apple gear,Positive emotion
9,Excited to meet the @samsungmobileus at #sxsw so I can show them my Sprint Galaxy S still running Android 2.1. #fail,Positive emotion


In [10]:
#covert the object data into string

In [11]:
data = data.astype('string')

In [12]:
data['emotion'].value_counts()

,count
emotion,
No emotion toward brand or product,5388
Positive emotion,2978
Negative emotion,570
I can't tell,156


In [13]:
data1= pd.get_dummies(data['emotion'],dtype='int')

In [14]:
data1

,I can't tell,Negative emotion,No emotion toward brand or product,Positive emotion
0,0,1,0,0
1,0,0,0,1
2,0,0,0,1
3,0,1,0,0
4,0,0,0,1
...,...,...,...,...
9087,0,0,0,1
9088,0,0,1,0
9089,0,0,1,0
9090,0,0,1,0


In [15]:
data1.columns = ['no_idea', 'negative', 'neutral','positive']
data1.head()

,no_idea,negative,neutral,positive
0,0,1,0,0
1,0,0,0,1
2,0,0,0,1
3,0,1,0,0
4,0,0,0,1


In [21]:
#train test split
x_train , x_test, y_train, y_test = train_test_split(data['tweet'],data1,test_size = 0.1 , random_state =0 , stratify=data1 )

In [22]:
x_test

,tweet
1142,ÛÏ@mention The 10 most dangerous IPhone apps. #sxsw {link}
4294,Bah! I somehow lost my little @mention iPad wipe- between sessions at #sxsw.
2720,Temp Apple store in Austin has quite a few #iPAD2 left... #SXSW
1602,True. RT @mention I bet the longest iPad lines are gonna be in Austin. #sxsw
8228,"I can just feel the buzz!RT @mention Google to Launch Major New Social Network Called Circles, Possibly Today {link} #sxsw"
...,...
271,Surprise! Apple has opened a pop-up store in Austin so that the nerds in town for #SXSW can get their new iPads. http:/ {link}
2132,The line at 6th &amp; Congress to enter the new Apple store wraps around 2 blocks #SXSW
2712,Get in the season with Clover Touch: {link} #Games #STPATRICK
5535,RT @mention Awesome new version of @mention for iPhone and new web site coming after #sxsw. Want a demo? Meet with @mention at #sxsw.


In [23]:
#vectorisation
vectorizer = TextVectorization(max_tokens=5000)


In [24]:
vectorizer.adapt(x_train)

In [25]:
#RNN
model = Sequential()

model.add(vectorizer)
model.add(Embedding(input_dim= 5000, output_dim=256,mask_zero=True))
model.add(SimpleRNN(128,activation='tanh',return_sequences = True))
model.add(SimpleRNN(64))

model.add(Dense(64,activation='tanh'))
model.add(Dense(4,activation='softmax'))

In [26]:
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [27]:
history = model.fit(x_train,y_train,
                    epochs = 10,
                    batch_size = 32,
                    validation_data = (x_test,y_test))

Epoch 1/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.5637 - loss: 0.9618 - val_accuracy: 0.6055 - val_loss: 0.9036
Epoch 2/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - accuracy: 0.7056 - loss: 0.7354 - val_accuracy: 0.6418 - val_loss: 0.9449
Epoch 3/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.8404 - loss: 0.4428 - val_accuracy: 0.6198 - val_loss: 1.0425
Epoch 4/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 21s 51ms/step - accuracy: 0.8813 - loss: 0.3142 - val_accuracy: 0.6209 - val_loss: 1.1137
Epoch 5/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - accuracy: 0.9133 - loss: 0.2255 - val_accuracy: 0.5923 - val_loss: 1.3243
Epoch 6/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.9258 - loss: 0.1803 - val_accuracy: 0.6176 - val_loss: 1.4422
Epoch 7/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - accuracy: 0.9292 - loss: 0.1591 - val_accuracy: 0.6231 - val_loss: 1.5505
Epoch 8/10
256/256 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.9351 - loss: 0.1514 - 